## Description

This example file propagates uncertainty through a model, using UQpy's RunModel module and an input file.

## Required files

To run this example, the following files must be present in the same directory as this Jupyter script:

1) matlab_model.py   
2) inpt.inp   
3) process_matlab_output.py

## Structure of files

1) "matlab_model.py" contains the python commands to run the model, which is a MATLAB file in this example    
2) "inpt.inp" is a template input file which is used to create input files for each of the simulations with sampled values for the probabilistic variables. This is a text file with placeholders for where the probabilistic variables must be inserted.    
3) "process_matlab_output.py" is a python script which processes the output. Within this script, if a function is used to process the output, this function should return the output quantity of interest. If a class is used to process the output, the quantity of interest must be saved as self.qoi within the class

In [ ]:
from UQpy.SampleMethods import STS
from UQpy.RunModel import RunModel
import os
import time

# Add MATLAB to $PATH - modify and use the command in the next line if necessary
# os.system("export PATH=$PATH:'/Applications/MATLAB_R2018a.app/bin'")

## Sample generation

In [ ]:
# Inputs to Stratified Sampling
sts_criterion = "centered"  # Samples are the center of the strata
# sts_criterion = "random"  # Samples are randomly drawn from within the strata
dist_name = "Uniform"  # Strata are uniformly distributed (i.e., of equal size) in a dimension. Can differ between
# dimensions

# Range of the variables
vmin1 = 0
vmax1 = 1
vmin2 = 0
vmax2 = 1
vmin3 = 0
vmax3 = 1

# Number of strata
ns1 = 3
ns2 = 1
ns3 = 1

# Build inputs to stratified sampling
dist_params = [[vmin1, vmax1], [vmin2, vmax2], [vmin3, vmax3]]
dimension = len(dist_params)
sts_design = [ns1, ns2, ns3]

# Call STS to generate samples
x_sts = STS(dimension=dimension, dist_name=dist_name, dist_params=dist_params, sts_design=sts_design,
            sts_criterion=sts_criterion)  # This creates the sample points.

## Calling RunModel

In [ ]:
#######################################################################################################################
# Inputs to RunModel
var_names = ['var1', 'var2', 'var3']

#######################################################################################################################
# Call to RunModel
t = time.time()
m = RunModel(samples=x_sts.samples, ntasks=1, model_script='matlab_model.py', input_template='inpt.inp',
             var_names=var_names, model_object_name="matlab", output_script='process_matlab_output.py',
             output_object_name='read_output', resume=False)
t_ser_matlab = time.time() - t
print("\nTime for serial execution:")
print(t_ser_matlab)